In [1]:
import numpy as np
import pandas as pd

In [2]:
crime_data = pd.read_csv('./crime_data_final.csv')

# Convert the 'Date' column to datetime
crime_data['Date'] = pd.to_datetime(crime_data['Date'])

# Find the latest date in the dataset
latest_date = crime_data['Date'].max()

print(f"The latest date in the dataset is: {latest_date}")

The latest date in the dataset is: 2025-02-02 00:00:00
